<a href="https://colab.research.google.com/github/murffps/machine-learning-titanic/blob/main/MLBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### Project Introduction ########
Titanic Data Prediction with Classification Supervised Learning : 

*   Prediction label for an individual as "survived" or "died"

*    take passenger/trip information and predict whether that passenger would survive:  

*   Is this a good ML question for good data for ML ? Why ? Why not? initial assumptions.
how to properly document this 
Cost Time trade offs  princples estimate
Business value ROI 
Success how to measure given the above https://github.com/mattharrison/ml_pocket_reference/blob/master/ch03.ipynb 

many diff sets of steps - pick one 


Algebreic Representation of Intent to create a function 
 y = f(x) to turn a feature into a label 
 x is a matrix, every column in x is a feature 
 the output y is a vector that contains labels(or targets) (classification uses labels or regression would use values) 

 



ML WORKFLOW STEPS 
Gather
clean 
Create feautres
normlaize 

sample data
testing data
training data
create model 
evaluate model - possible change question
deploy model

The CRoss Industry Standard Process for Data Mining (CRISP-DM) is a process model that serves as the base for a data science process. It has six sequential phases:

Business understanding – What does the business need?
Data understanding – What data do we have / need? Is it clean?
Data preparation – How do we organize the data for modeling?
Modeling – What modeling techniques should we apply?
Evaluation – Which model best meets the business objectives?
Deployment – How do stakeholders access the results?
Published in 1999 to standardize data mining processes across industries, it has since become the most common methodology for data mining, analytics, and data science projects.

Data science teams that combine a loose implementation of CRISP-DM with overarching team-based agile project management approaches will likely see the best results.

SOURCES:
https://www.datascience-pm.com/crisp-dm-2/


https://web.archive.org/web/20220401041957/https://www.the-modeling-agency.com/crisp-dm.pdf


https://www.oreilly.com/library/view/machine-learning-pocket/9781492047537/
Book sections
Introduction
Overview of the Machine Learning Processing
Classification Walkthrough: Titanic Dataset
Missing Data
Cleaning Data
Exploring
Preprocess Data
Feature Selection
Imbalanced Classes
Classification
Model Selection
Metrics and Classification Evaluation
Explaining Models
Regression
Metrics and Regression Evaluation
Explaining Regression Models
Dimensionality Reduction
Clustering
Pipelines


I. Business Understanding



TDSP (Team Data Science Process): 2 steps also taken 

  Data Acquisition & Understanding (Knowledge Information Patterns)



  
  Afetr evaulatio Customer Acceptance



In [10]:
# create requirements.txt with required python code packages
%%writefile requirements.txt
matplotlib
pandas
sklearn 
yellowbrick

Overwriting requirements.txt


In [ ]:
# install 
!pip install -r requirements.txt
!pip install gwpy --quiet #supposed to supress logs

# New Section

In [ ]:
# import tools and libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import (ensemble, preprocessing,tree)
from sklearn.metrics import (auc, confusion_matrix, roc_auc_score, roc_curve)
from sklearn.model_selection import (train_test_split, StratifiedKFold)
from yellowbrick.classifier import (ConfusionMatrix, ROCAUC)
from yellowbrick.model_selection import (LearningCurve)
# nice way to use profiling in colab notebooks 
import sys 
!{sys.executable} -m pip install -U ydata-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension
# https://ydata-profiling.ydata.ai/docs/master/pages/getting_started/installation.html
from ydata_profiling import ProfileReport
import IPython

In [13]:
# GATHER DATA - load an excel file
url = ("https://biostat.app.vumc.org/wiki/pub/Main/DataSets/titanic3.xls")
df = pd.read_excel(url) # create a data frame (matrix of data) with column labels (as opposed to numpy array)
orig_df = df 

### PREPARE AND CLEAN DATA 
# 1. strategy to determine data readiness and quality for use
# 2. hanlde the  issues found, once quality or readiness is determined CLEAN DATA 

#### Definitions and Notes and Guide on above:
##### extended note on data types : look for pandas friendly data types to store data--> int64 float64 datetime64[ns] object (combination of string and other). 
##### pandas will try to coerce data and fall back to object, convert date and strings to numeric or feature engineer one. (cardinality h/l and dummy columns?)
##### standardizing: some models perform better if they are standardized by given a mean value of 0 and a standard deviation of 1
##### leaky features (info that are not available for new data yet or cheating by giving data to the model)
##### use count for checking for missing data - only includes values not NaN - 

---

SME can help to know what to do in case of ..

---
Data 
There are two primary types of numerical data: discrete and continuous data.
- continuous refers to data that can be measured.
- discrete also referred to as discrete values, is data that only takes certain values. Commonly in the form of whole numbers or integers, this is data that can be counted and has a finite number of values. These values must be able to fall within certain classifications and are unable to be broken down into smaller parts.
median vs mode vs mean 
many exploratory methods and analytical approaches only work with specific data types. For this reason, being able to determine the nature of your data will make handling your data more manageable and effective when it comes to yielding timely insights.

In [14]:
# inspect #rows columns & summary stats
df.shape

(1309, 14)

In [15]:
# snapshot 
df.describe().iloc[:, :2]

,pclass,survived
count,1309.000000,1309.000000
mean,2.294882,0.381971
std,0.837836,0.486055
min,1.000000,0.000000
25%,2.000000,0.000000
50%,3.000000,0.000000
75%,3.000000,1.000000
max,3.000000,1.000000


In [16]:
# check for type of values like integer or float, NA or missing data, outliers, leaky features 
df.dtypes

pclass         int64
survived       int64
name          object
sex           object
age          float64
sibsp          int64
parch          int64
ticket        object
fare         float64
cabin         object
embarked      object
boat          object
body         float64
home.dest     object
dtype: object

In [17]:
# a. analyze or spot check data for format we can use  ie min max, 
df.isnull().sum() # one way to get a count of missing data in each column 

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

In [18]:
# axis 0 means along the index so apply the operation on said axis 
df.isnull().sum(axis=1).loc[:10] 

0     1
1     1
2     2
3     1
4     2
5     1
6     1
7     2
8     1
9     2
10    1
dtype: int64

In [19]:
# get the percentage of null values
df.isnull().mean() * 100 

pclass        0.000000
survived      0.000000
name          0.000000
sex           0.000000
age          20.091673
sibsp         0.000000
parch         0.000000
ticket        0.000000
fare          0.076394
cabin        77.463713
embarked      0.152788
boat         62.872422
body         90.756303
home.dest    43.086325
dtype: float64

In [20]:
#  inspect at rows with missing data by using a Boolean array 
mask = df.isnull().any(axis=1)
mask.head()  # rows
# data.columns = data.columns.str.strip() handle white space 


0    True
1    True
2    True
3    True
4    True
dtype: bool

In [21]:
df[mask].body.head()

0      NaN
1      NaN
2      NaN
3    135.0
4      NaN
Name: body, dtype: float64

In [22]:

# f. profile data 
profile = ProfileReport(df, title="titanic data set", html={'style' : {'full_width':True}})
profile.to_file(output_file="file.html")
# View HTML profile report
IPython.display.HTML(filename='/content/file.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Number of variables,14
Number of observations,1309
Missing cells,3855
Missing cells (%),21.0%
Duplicate rows,0
Duplicate rows (%),0.0%
Total size in memory,143.3 KiB
Average record size in memory,112.1 B
Categorical,7
Numeric,5
Unsupported,2


In [24]:
name = df.name
name.head(3)

0     Allen, Miss. Elisabeth Walton
1    Allison, Master. Hudson Trevor
2      Allison, Miss. Helen Loraine
Name: name, dtype: object

In [ ]:
# box plot or histogram to spot outliers  - manual or niche spot graphs possible
#  what needs to meet outside of the profile report ? 
# Findings from Profile Report ✨ finding issues vs trends suspicions... to start ... learning 
# ready to move on to cleaning ?

In [ ]:
# Handle Outliers(check min/max, ) seems like the most common sense while in the graphs plots before the code ones 
# https://www.udemy.com/course/data-science-and-machine-learning-with-python-hands-on/learn/lecture/4020118?start=315#overview

In [ ]:
# CLEAN (vs PREPARE? is prepare after import or import a certain way ) STEPS  
# Standardize 
# Normalize

In [ ]:
# a. impute -  refers to the procedure of using alternative values in place of missing data. Like an estimate or other instead of deletion.
# b. to replace deal with missing or bad data: 0's is bad data
# sibsp has 891 (68.1%) zeros	
# parch has 1002 (76.5%) zeros	
# fare has 17 (1.3%) zeros	




S      914
C      270
Q      123
NaN      2
Name: embarked, dtype: int64

In [ ]:
# c. Use S the most common to  


In [ ]:
# Handle Missing Data
# (check for NaN, null) (a-e options)
# age has 263 (20.1%) missing values	
# cabin has 1014 (77.5%) missing values	
# boat has 823 (62.9%) missing values	
# body has 1188 (90.8%) missing values	
# home.dest has 564 (43.1%) missing values

In [ ]:
# d. use pandas to create dummy columns with all 0s 
df = pd.get_dummies(df)
df.columns

In [ ]:
# e. drop them -- One possible strategy is to drop columns with 
     # reasons for dropping: no variance or signal or perfect or very high positive or negative correlation
# age (interpolate values), 
# cabin
# boat - dtype not supported issues 
# body missing data in many rows and leaks data (gives survived or deceased)
df = df.drop(
    columns=[
        "name",
        "ticket",
        "home.dest",
        "boat",
        "body",  
        "cabin" 
    ]) 

df.sex.value_counts(dropna=False)
df.embarked.value_counts(dropna=False)
df = df.drop(columns="sex_male")
# OR df = pd.get_dummies(df, drop_first=True)
df.columns
y = df.survived
X = df.drop(columns="survived")

0     Allen, Miss. Elisabeth Walton
1    Allison, Master. Hudson Trevor
2      Allison, Miss. Helen Loraine
Name: name, dtype: object

In [ ]:
## CREATE FEATURES 



In [ ]:
## SAMPLE DATA 
# - pull out 30% 
# X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=42)
#not working NameError: name 'model_selection' is not defined

# Impute missing values for age on training set 
# clean training set now ?